# Ejemplo 1: Proyeccion de cuentas

## El modelo a emplear sera Holt - Winter con factor multiplicativo


1. Cargar y preparar los datos

Antes de aplicar cualquier modelo, es fundamental asegurar la calidad de los datos y la correcta transformación de la fecha.

In [ ]:
# en la terminal colocar:
#  pip install openpyxl
# 


import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel("Datos.xlsx")  

# Transformar la columna de fecha a formato datetime
df['Year'] = pd.to_datetime(df['Year'])  
df.set_index('Year', inplace=True)  


In [ ]:
# veamos que los elementos esten cargados correctamentes
df.head()

In [ ]:
# ahora veamos los estadisticos historicos

df.describe()
# print(df.describe())

2. Análisis exploratorio inicial

Antes de modelar, debemos visualizar los datos y verificar posibles problemas.

In [ ]:
import matplotlib.pyplot as plt

# Graficar la serie original
plt.figure(figsize=(10,5))
plt.plot(df.index, df['Observ- ations'], label="Serie Original", color='blue')
plt.xlabel("Fecha")
plt.ylabel("Observ- ations")
plt.title("Serie de Tiempo Original")
plt.legend()
plt.grid()
plt.show()


3. Verificación de estacionariedad

La prueba de Dickey-Fuller nos ayuda a determinar si la serie es estacionaria o si requiere transformación.

In [ ]:
# en la consola colocar :
# pip install statsmodels

from statsmodels.tsa.stattools import adfuller

# Aplicar prueba de Dickey-Fuller
resultado_adf = adfuller(df['Observ- ations'])
print(f"ADF Statistic: {resultado_adf[0]}")
print(f"p-value: {resultado_adf[1]}")

# Interpretación:
if resultado_adf[1] < 0.05:
    print("La serie es estacionaria.")
else:
    print("La serie NO es estacionaria.")


4. Descomposición de la serie de tiempo

Para entender tendencias y estacionalidad, aplicamos una descomposición multiplicativa.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Descomposición de la serie
descomposicion = seasonal_decompose(df['Observ- ations'], model='multiplicative', period=12)
descomposicion.plot()
plt.show()



5. Aplicar el modelo Holt-Winters

Ahora ajustamos el modelo multiplicativo, que captura tanto tendencia como estacionalidad.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Ajustar el modelo Holt-Winters multiplicativo
modelo = ExponentialSmoothing(df['Observ- ations'], trend='multiplicative', seasonal='multiplicative', seasonal_periods=12)
ajuste = modelo.fit()

# Agregar las estimaciones al DataFrame
df['Estimación'] = ajuste.fittedvalues


6. Generar proyecciones

Definimos un horizonte de días y obtenemos las predicciones.

In [ ]:
# Número de días a proyectar
horizonte = 10  
predicciones = ajuste.forecast(horizonte)

# Crear DataFrame con las predicciones
df_predicciones = pd.DataFrame({
    'Fecha': pd.date_range(start=df.index[-1], periods=horizonte+1, freq='D')[1:],
    'Valor Estimado': predicciones
})


In [ ]:
# Observación de las primeras cinco predicciones

df_predicciones.head()

7. Visualización final

Graficamos la serie original junto con los valores ajustados.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

# Gráfico de la serie original
plt.plot(df.index, df['Observ- ations'], label="Histórico", color='blue')

# Gráfico de los valores ajustados por el modelo Holt-Winters
plt.plot(df.index, df['Estimación'], label="Ajuste del Modelo", color='orange', linestyle='dashed')

# Configuración del gráfico
plt.xlabel("Fecha")
plt.ylabel("Valor")
plt.title("Serie de Tiempo - Datos Originales vs. Ajuste Holt-Winters con factor multiplicativo")
plt.legend()
plt.grid()
plt.show()


Ahora graficaremos los valores originales con las respectivas proyecciones a un horizonte de 10 meses

In [ ]:
plt.figure(figsize=(10,5))

# Gráfico de la serie original
plt.plot(df.index, df['Observ- ations'], label="Histórico", color='blue')

# Gráfico de las proyecciones
plt.plot(df_predicciones['Fecha'], df_predicciones['Valor Estimado'], label="Predicción", color='red', linestyle='dashed')

# Configuración del gráfico
plt.xlabel("Fecha")
plt.ylabel("Observ- ations")
plt.title("Serie de Tiempo - Datos Originales y Predicciones")
plt.legend()
plt.grid()
plt.show()


8. Veamos los parametros del modelo

In [ ]:
# Mostrar parámetros del modelo Holt-Winters
print("Parámetros del modelo:")
print(ajuste.params)


In [ ]:
print("Nivel de suavización:", ajuste.params['smoothing_level'])
print("Tendencia de suavización:", ajuste.params['smoothing_trend'])
print("Estacionalidad de suavización:", ajuste.params['smoothing_seasonal'])


9. Exportar resultados a Excel

Guardamos los datos originales y las proyecciones en dos pestañas.

In [ ]:
# escribir en la terminal:
# pip install xlsxwriter

with pd.ExcelWriter("resultado.xlsx", engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name="Datos Originales")
    df_predicciones.to_excel(writer, sheet_name="Proyecciones")
